# Loading Trained Model

In [77]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from src.Mymodel import MyResNet34
from src.Mymodel import MyResNet_CIFAR
from src.Mydataloader import LoadDataset
from src.Mytraining import DoTraining

In [78]:
"""Dataset selection"""
DATASET = "CIFAR10"
# DATASET = "CIFAR100"
# DATASET = "ImageNet2012"

"""Model selection for CIFAR"""
NUM_LAYERS_LEVEL = 5

"""Dataset parameters"""
BATCH = 128
SHUFFLE = True
NUMOFWORKERS = 8
PIN_MEMORY = True
SPLIT_RATIO = 0.9

"""optimizer parameters"""
OPTIMIZER = "SGD"
# OPTIMIZER = "Adam"
# OPTIMIZER = "Adam_decay"


file_path = ""
if DATASET == "ImageNet2012":
    file_path = f"MyResNet34_{BATCH}_{OPTIMIZER}"
    _model_name = f"MyResNet34_{DATASET}_{BATCH}_{OPTIMIZER}"

    # casenum = 1
    # if casenum != 0:
    #     _model_name += f"_case{casenum}"
else:
    file_path = f"MyResNet{NUM_LAYERS_LEVEL*6+2}_{BATCH}_{OPTIMIZER}"
    _model_name = f"MyResNet{NUM_LAYERS_LEVEL*6+2}_{DATASET}_{BATCH}_{OPTIMIZER}"

if SPLIT_RATIO != 0:
    _model_name += f"_{int(SPLIT_RATIO*100)}"
    file_path += f"_{int(SPLIT_RATIO*100)}"

In [79]:
tmp = LoadDataset(root="data", seceted_dataset=DATASET, split_ratio=SPLIT_RATIO)
_, valid_data, test_data, COUNT_OF_CLASSES = tmp.Unpack()

if valid_data is not None:
    valid_dataloader = DataLoader(
        valid_data,
        batch_size=BATCH,
        shuffle=SHUFFLE,
        num_workers=NUMOFWORKERS,
        pin_memory=PIN_MEMORY,
        # pin_memory_device="cuda",
        persistent_workers=True,
    )
    print("valid.transforms =", valid_data.transform, valid_dataloader.batch_size)
else:
    valid_dataloader = None

if test_data is not None:
    test_dataloader = DataLoader(
        test_data,
        batch_size=BATCH,
        shuffle=SHUFFLE,
        num_workers=NUMOFWORKERS,
        pin_memory=PIN_MEMORY,
        # pin_memory_device="cuda",
        persistent_workers=True,
    )
    print("test.transforms =", test_data.transform, test_dataloader.batch_size)
else:
    test_dataloader = None

"""Model selection"""
if DATASET == "CIFAR10":
    eval_dataloader = test_dataloader
elif DATASET == "ImageNet2012":
    eval_dataloader = valid_dataloader

print(len(eval_dataloader))

/home/lee/anaconda3/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


-----------------------------------------------------------------------
Dataset :  CIFAR10
- Length of Train Set :  45000
- Length of Valid Set :  5000
- Length of Test Set :  10000
- Count of Classes :  10
-----------------------------------------------------------------------
valid.transforms = ToTensor() 128
test.transforms = ToTensor() 128
79


In [80]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if DATASET == "ImageNet2012":
    model = MyResNet34(num_classes=COUNT_OF_CLASSES, Downsample_option="B").to(device)

else:
    model = MyResNet_CIFAR(
        num_classes=COUNT_OF_CLASSES, num_layer_factor=NUM_LAYERS_LEVEL
    ).to(device)



model.load_state_dict(torch.load(f"models/{_model_name}/{file_path}.pth"))

<All keys matched successfully>

In [81]:
testingclass = DoTraining(
    model=model, criterion=nn.CrossEntropyLoss(), optimizer=None, device=device
)
test_loss, test_acc = testingclass.Forward_eval(eval_dataloader)

print("test_loss:", test_loss)
print(f"test_acc: {test_acc*100:.2f}%")
print(f"test_error: {100 - test_acc*100:.2f}%")
print(file_path)

test_loss: 0.5532871841252605
test_acc: 83.39%
test_error: 16.61%
MyResNet32_128_SGD_90
